In [1]:
!pip install sacremoses --q

In [13]:
import os
import re
import sys
import unicodedata
import pandas as pd
from sacremoses import MosesPunctNormalizer

In [3]:
FOLDER_PATH = '/dataset/'
FILE_NAME = 'Tales'
DATA_SPLIT = ['train', 'valid', 'test']
DATA_LANGUAGES  = {
    'ro': 'Romanian',
    'rup': 'Romanian_RUP',
    'rup_std': 'Romanian_RUP_STD',
    'rup_cun': 'Romanian_RUP_CUN',
    'en': 'English',
    'es': 'Spanish',
    'fr': 'French'
}

In [5]:
def read_file(file_path):
    with open(file_path) as file:
        return list(filter(lambda x: len(x), file.read().split('\n')))

In [6]:
def load_language_from_file(language):
    df = pd.DataFrame()
    for split in DATA_SPLIT:
        file_path = os.path.join(FOLDER_PATH, f"{FILE_NAME}.{split}.{language}")
        if os.path.isfile(file_path):
            new_df = pd.DataFrame({
                language: read_file(file_path),
                'split': split
            })
            df = pd.concat([df, new_df], ignore_index=True)
            
    return df

In [7]:
def build_translations_dataframe(languages):
    df = pd.DataFrame()
    for language in languages:
        new_df = load_language_from_file(language)
        if 'split' in df:
            df = df.drop('split', axis=1)
        df = pd.concat([df, new_df], axis=1)
        
    return df

In [65]:
translations_df = build_translations_dataframe(DATA_LANGUAGES.keys())
translations_df

,ro,rup,rup_std,rup_cun,en,es,fr,split
0,TEORIA ȘI PRACTICA,TEORIA ȘI PRACTICA,TEORIA ȘI PRACTICA,TEORIA ShI PRACTICA,THEORY AND PRACTICE,TEORÍA Y PRÁCTICA,THÉORIE ET PRATIQUE,train
1,„Sunt unele deprinderi pe care le câștigi doar...,"„Sănt îndoauă învețuri, cari si amintă mași cu...","„Sănt îndoauă învețuri, cari si amintă mași cu...","„Sãnt ãndoauã ãnvetsuri, cari si amintã mashi ...","""There are some skills you gain just by readin...","""Hay algunas habilidades que se adquieren con ...","""Il y a des compétences que l'on acquiert simp...",train
2,Așa grăia într-un rând un bătrân înțelept - că...,"Ași zbura unoară un auș cu minte, cătră-ndoi t...","Ași zbura unoară un auș cu minte, cătră-ndoi t...","Ashi zbura unoarã un aush cu minte, cãtrã-ndoi...",So said a wise old man once - to young people ...,Así se lo dijo una vez un viejo sabio a unos j...,C'est ce qu'a dit un jour un vieux sage à des ...,train
3,"Ca să pricepeți ce vă spusei până aici, luați ...","Ca s-lă da s-achicăsească ți va dzîșu pîn aua,...","Ca s-lă da s-achicăsească ți va d̦îșu pîn aua,...",Ca s-lã da s-achicãseascã tsi va dzãshu pãn au...,"To understand what I have told you so far, con...",Para comprender lo que te he dicho hasta ahora...,Pour comprendre ce que je vous ai dit jusqu'à ...,train
4,Cică un împărat dintre cei mai înțelepți - și ...,Un amiră cu multe dhrăñi în cap - și alăgat că...,Un amiră cu multe dhrăńi în cap - și alăgat că...,Un amirã cu multe dhrãnji ãn cap - shi alãgat ...,"Some wise emperor - and a wise man, not a joke...","Algún sabio emperador -y un sabio, no un chist...","Un sage empereur - et un sage, ce n'est pas un...",train
...,...,...,...,...,...,...,...,...
2106,Vântul le mângâia lin obrajii... Parcă-i căinà...,Vimtul lișor lă hărisi fața... și-l’i plîmse.....,Vimtul lișor lă hărisi fața... și-ľi plîmse......,Vimtul lishor lã hãrisi fatsa... shi-lji plãms...,The wind gently caressed their cheeks... It's ...,El viento acariciaba suavemente sus mejillas.....,Le vent caresse doucement leurs joues... C'est...,test
2107,Lumina se revarsă peste sat. Dinu și Vira stau...,Hoara s-avea umplută di luñină. Dina cu Vira ș...,Hoara s-avea umplută di luńină. Dina cu Vira ș...,Hoara s-avea umplutã di lunjinã. Dina cu Vira ...,Light pours over the village. Dinu and Vira ar...,La luz se derrama sobre la aldea. Dinu y Vira ...,La lumière se répand sur le village. Dinu et V...,test
2108,"La dangătul clopotului, satul răsuna - pustiu....","La boațea aistă, hoara arăsuna jilos... și-apo...","La boațea aistă, hoara arăsuna jilos... și-apo...","La boatsea aistã, hoara arãsuna jilos... shi-a...","At the ringing of the bell, the village rang -...","Al toque de campana, el pueblo sonó - desierto...","Au son de la cloche, le village retentit - dés...",test
2109,"Vira se întoarse acasă... Cânta, râdea... Își ...","Vira s-turnă acasă... cîntă, arăse... s-alăxi ...","Vira s-turnă acasă... cîntă, arăse... s-alăxi ...","Vira s-turnã acasã... cãntã, arãse... s-alãxi ...","Vira returned home... Singing, laughing... She...","Vira volvió a casa... Cantando, riendo... Se p...","Vira est rentrée chez elle... Elle chante, ell...",test


In [69]:
def get_non_printing_char_replacer(replace_by: str = " "):
    non_printable_map = {
        ord(c): replace_by
        for c in (chr(i) for i in range(sys.maxunicode + 1))
        if unicodedata.category(c) in {"C", "Cc", "Cf", "Cs", "Co", "Cn"}
    }

    def replace_non_printing_char(text) -> str:
        return text.translate(non_printable_map)

    return replace_non_printing_char

replace_nonprint = get_non_printing_char_replacer("")

In [81]:
def process_text(text, language):
    mpn = MosesPunctNormalizer(lang=language)
    mpn.substitutions = [(re.compile(r), sub) for r, sub in mpn.substitutions]
    
    text = text.lower()
    
    clean = mpn.normalize(text)
    clean = replace_nonprint(clean)
    # ex: replace '𝓕𝔯𝔞𝔫𝔠𝔢𝔰𝔠𝔞' with 'Francesca'
    clean = unicodedata.normalize("NFKC", clean)
    clean = clean.replace('"', '').replace(' - ', ' ')
    clena = clean.strip()
    
    return clean

In [82]:
def apply_text_processing(df):
    language_map = {
        'ro': 'ro',
        'rup': 'ro',  
        'rup_std': 'ro',  
        'rup_cun': 'ro',
        'en': 'en',
        'es': 'es',
        'fr': 'fr'
    }
    
    for column, lang in language_map.items():
        df[column] = df[column].apply(lambda text: process_text(text, lang))

    return df

In [83]:
processed_df = apply_text_processing(translations_df)
processed_df

,ro,rup,rup_std,rup_cun,en,es,fr,split
0,teoria și practica,teoria și practica,teoria și practica,teoria shi practica,theory and practice,teoría y práctica,théorie et pratique,train
1,sunt unele deprinderi pe care le câștigi doar ...,"sănt îndoauă învețuri, cari si amintă mași cu ...","sănt îndoauă învețuri, cari si amintă mași cu ...","sãnt ãndoauã ãnvetsuri, cari si amintã mashi c...",there are some skills you gain just by reading...,hay algunas habilidades que se adquieren con s...,il y a des compétences que l'on acquiert simpl...,train
2,așa grăia într-un rând un bătrân înțelept cătr...,"ași zbura unoară un auș cu minte, cătră-ndoi t...","ași zbura unoară un auș cu minte, cătră-ndoi t...","ashi zbura unoarã un aush cu minte, cãtrã-ndoi...",so said a wise old man once to young people wh...,así se lo dijo una vez un viejo sabio a unos j...,c'est ce qu'a dit un jour un vieux sage à des ...,train
3,"ca să pricepeți ce vă spusei până aici, luați ...","ca s-lă da s-achicăsească ți va dzîșu pîn aua,...","ca s-lă da s-achicăsească ți va d̦îșu pîn aua,...",ca s-lã da s-achicãseascã tsi va dzãshu pãn au...,"to understand what i have told you so far, con...",para comprender lo que te he dicho hasta ahora...,pour comprendre ce que je vous ai dit jusqu'à ...,train
4,cică un împărat dintre cei mai înțelepți și um...,un amiră cu multe dhrăñi în cap și alăgat căt ...,un amiră cu multe dhrăńi în cap și alăgat căt ...,un amirã cu multe dhrãnji ãn cap shi alãgat cã...,"some wise emperor and a wise man, not a joke s...","algún sabio emperador -y un sabio, no un chist...","un sage empereur et un sage, ce n'est pas une ...",train
...,...,...,...,...,...,...,...,...
2106,vântul le mângâia lin obrajii... parcă-i căinà...,vimtul lișor lă hărisi fața... și-l'i plîmse.....,vimtul lișor lă hărisi fața... și-ľi plîmse......,vimtul lishor lã hãrisi fatsa... shi-lji plãms...,the wind gently caressed their cheeks... it's ...,el viento acariciaba suavemente sus mejillas.....,le vent caresse doucement leurs joues... c'est...,test
2107,lumina se revarsă peste sat. dinu și vira stau...,hoara s-avea umplută di luñină. dina cu vira ș...,hoara s-avea umplută di luńină. dina cu vira ș...,hoara s-avea umplutã di lunjinã. dina cu vira ...,light pours over the village. dinu and vira ar...,la luz se derrama sobre la aldea. dinu y vira ...,la lumière se répand sur le village. dinu et v...,test
2108,"la dangătul clopotului, satul răsuna pustiu......","la boațea aistă, hoara arăsuna jilos... și-apo...","la boațea aistă, hoara arăsuna jilos... și-apo...","la boatsea aistã, hoara arãsuna jilos... shi-a...","at the ringing of the bell, the village rang d...","al toque de campana, el pueblo sonó desierto.....","au son de la cloche, le village retentit déser...",test
2109,"vira se întoarse acasă... cânta, râdea... își ...","vira s-turnă acasă... cîntă, arăse... s-alăxi ...","vira s-turnă acasă... cîntă, arăse... s-alăxi ...","vira s-turnã acasã... cãntã, arãse... s-alãxi ...","vira returned home... singing, laughing... she...","vira volvió a casa... cantando, riendo... se p...","vira est rentrée chez elle... elle chante, ell...",test


In [84]:
processed_df.to_csv("Tales_processed.csv");